# Importar librerías

In [ ]:
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installatio

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

# Descargar y leer el dataset

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
path_to_file = tf.keras.utils.get_file("shakespeare.txt", dataset_url)

1115394/1115394 [==============================] - 0s 0us/step


# Leer y explorar el texto

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')
#imprimimos los primeros 500 caracteres del texto:
print(f'First 500 characters:\n{text[:500]}')

Length of text: 1115394 characters
First 500 characters:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


# Modelo carácter a carácter

## 1. Preprocesamiento

In [ ]:
#Se extraen los carácteres únicos del texto con el objetivo de construir el vocabulario.
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [ ]:
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')
chars

<tf.Tensor: shape=(1115394,), dtype=string, numpy=array([b'F', b'i', b'r', ..., b'g', b'.', b'\n'], dtype=object)>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(1115394,), dtype=string, numpy=array([b'F', b'i', b'r', ..., b'g', b'.', b'\n'], dtype=object)>

In [ ]:
#convertimos los índices a texto
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Convertir texto en IDs y generar secuencias

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
#Dataset en base a los índices númericos del texto
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
#primeros 10 índices convertidos a caracteres
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100

In [ ]:
#establecemos secuencias de longitud fija
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
#secuencia en dos partes;
#entrada: secuencia sin el último carácter, y objetivo: secuencia sin el primer carácter.
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
;split_input_target(list("Julián es un capo"))

(['J',
  'u',
  'l',
  'i',
  'á',
  'n',
  ' ',
  'e',
  's',
  ' ',
  'u',
  'n',
  ' ',
  'c',
  'a',
  'p'],
 ['u',
  'l',
  'i',
  'á',
  'n',
  ' ',
  'e',
  's',
  ' ',
  'u',
  'n',
  ' ',
  'c',
  'a',
  'p',
  'o'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## 2. Construcción del modelo

Se va a contruir, entrenar y utilizar un modelo de red neuronal recurrente (RNN) para predecir texto de forma secuencial dónde se predice el siguiente carácter dado un conjunto de caracteres previos.

In [ ]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Longitud del vocabulario en la capa StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensión del embedding
embedding_dim = 256

# Número de unidades RNN
rnn_units = 1024

Modelo de red neuronal

In [ ]:

class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
      x = self.embedding(inputs)
      if states is None:
        states = self.gru.get_initial_state(x)
      x, states = self.gru(x, initial_state=states, training=training)
      x = self.dense(x, training=training)

      if return_state:
        return x, states
      else:
        return x


In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([36, 13, 61, 45, 57, 25, 47, 64, 35, 47, 65, 57, 41, 45, 29, 20, 58,
        1, 19, 31, 23, 43, 61, 64, 61, 43, 46,  4, 36, 24, 11, 36, 44, 43,
       41, 51, 27, 34, 31, 60, 48, 62, 48, 35, 41, 62, 25, 41, 39, 44, 26,
       65, 56, 47, 56, 59, 25, 52, 59,  6, 63, 28, 10, 53, 64, 17, 34, 36,
        6, 61, 10, 16, 46, 41, 59, 44, 49, 36, 33, 18, 36, 25, 32,  4, 57,
        4, 29, 31, 12, 16, 56, 26, 15, 32, 28, 36,  8, 51, 55, 52])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'KE:\nThy pains, Fitzwater, shall not be forgot;\nRight noble is thy merit, well I wot.\n\nHENRY PERCY:\nT'

Next Char Predictions:
 b"W?vfrLhyVhzrbfPGs\nFRJdvyvdg$WK:WedblNURuiwiVbwLbZeMzqhqtLmt'xO3nyDUW'v3CgbtejWTEWLS$r$PR;CqMBSOW-lpm"


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1909533, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.085754

Entrenamiento del modelo

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directorio donde guardo los checkpoint
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)  # Crea el directorio si no existe

#Nombre de los archivos de checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

#guardo los pesos del modelo
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=False
)

#para evitar overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',          # O 'val_loss' si tienes validación
    patience=5,              # Número de épocas sin mejora antes de detener
    restore_best_weights=True # Restaura los pesos del mejor modelo
)


In [ ]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, early_stopping])

Epoch 1/100
172/172 [==============================] - 15s 71ms/step - loss: 2.7278
Epoch 2/100
172/172 [==============================] - 14s 72ms/step - loss: 1.9942
Epoch 3/100
172/172 [==============================] - 14s 72ms/step - loss: 1.7194
Epoch 4/100
172/172 [==============================] - 14s 73ms/step - loss: 1.5583
Epoch 5/100
172/172 [==============================] - 15s 75ms/step - loss: 1.4589
Epoch 6/100
172/172 [==============================] - 18s 74ms/step - loss: 1.3904
Epoch 7/100
172/172 [==============================] - 14s 72ms/step - loss: 1.3376
Epoch 8/100
172/172 [==============================] - 15s 74ms/step - loss: 1.2929
Epoch 9/100
172/172 [==============================] - 15s 80ms/step - loss: 1.2529
Epoch 10/100
172/172 [==============================] - 14s 73ms/step - loss: 1.2121
Epoch 11/100
172/172 [==============================] - 15s 81ms/step - loss: 1.1737
Epoch 12/100
172/172 [==============================] - 16s 75ms/step - lo

Genero texto con temperatura

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)


#se genera un carácter
  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model_02 = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.2)
one_step_model_05 = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.5)
one_step_model_07 = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.7)
one_step_model_10 = OneStep(model, chars_from_ids, ids_from_chars)
one_step_model_15 = OneStep(model, chars_from_ids, ids_from_chars, temperature=1.5)
one_step_model_30 = OneStep(model, chars_from_ids, ids_from_chars, temperature=3)

- temp 0.2

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_02.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
Ye'rayed his enemies,
Recove his broken stronger by him.
Farewell, sweet whensway: believe me, and speak not straight,
Where the northern lames are up, I treach you so it,
It is no sentinuse makes yourself.

CORIOLANUS:
Have you voices?

FRAAN:
God save your loyal sigh!--
Do, good my lord, and leave us here,
And make my wars on my true and like a father's death,
And not by love, and I'll be her birth.

KING RICHARD II:
The charge I bore half my desire yet instruction
may sigh and still appear go to your grace,
The latest not speak with men allow,
I never indeed had send the adventure
Where nothing can be in readiness.

NORSOLIO:
What is the matter?

DUKE OF YORK:
Perishon, madam: there's some coats blush'd unto their news.

GRUMIO:
Ay, marroast; and, as I said, and a half,
Or if you lie done words; I see Queen Margaret, as the way before.

KING RICHARD II:
We will ourselves whilst you have charged a king of
breath; there is some wandering that the duty where
The end of s

- temp 0.5

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_05.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
Ye're lords;
But let him swear at your suit of farches blood!
My prayers are my dreams; who would pluck him
to pieces. 'Parcest Master Froth: fame, it is Aufidius,
When this is were distinuted: he said he would
May stand alive; and then to give them would
Condition of streaks, and come with thee my fleet,
To make a meeting through the sacrament,
To strive to blazed, or wrongfully and usurps
always grant to sund. My track men faith,
I'll tell you now, see that attain it.

DUCHESS OF YORK:
Art thou my sail! that I have no cause
I may breeve not of the dire; and as he says,
She shall be married to noon out of their arms.

DUKE OF AUMERLE:
I brought high Hereford, hate thy crown.

WINCENCIOLINA:
What must be, whom I do set it on?
As son and was a madm that hath the right!
Call it a truth words.

EXTON:
'Halk'st of Burgundy, welcome!

AUFIDIUS:
I will practed him of perporation:
A boy make me browled it; and what thou hast not speak too line
That have been absence of them but

- temp 0.7

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_07.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
Ye'ry to Signifry!

CAPULET:
What not, that's sweat? Neven where have I done with her.
Put thy lady bones, to do your metely,
The ceremonious night I am so break, to pity,
He'll frown of wall: I end so must thy came yourself
Have almost stoly hated.

GLOUCESTER:
Harp! how my deeds did shall be to soon compassion;
All men of England whereal my daughter come;
Whilst thou looks are angels like deeds deserving
To bear a passing course to bear us.

HERMIONE:
Should a villain say so,
The conquest of you the causes of four-scope:
Which once they are coming to you that I parted
Bit him here affect his wife and marry his mercy.

QUEEN ELIZABETH:
My heart is great deplain? All this in them,
And thou, Lord Oxford, would the western flight,
Appoints my conscience and to help you, if
I may not be tedious; and my promise
known with counters and my name in love;
The fruit-crown fled of restrice, which are here by my charity.

QUEEN ELIZABETH:
To fearful not set upon a noble demand.
Had

- temp 1

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_10.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
Ye'ry we'll haste you. You pity he
to death, a dozard to command
And do we bound you where her early days:
The bloody day is not for our falsehood to thyself
And already his countenances, gives it at
Your own.

ALONSO:
Is it is; yet then rejuies that less vower like a league thine well?
She meet, my lord, I'll pract myself
To trust to make a monarch'd with to slow,
To speak by with me; get thee unease,
Too power the match if they do not let me speak.

WARWICK:
I must be content to swear: it not beloved
What I saw sund. O, that she did low well believe
Henceforth Citizens:
An easy task it; and it was by ta'en for
His majesty.

First Senator:
My general-case,
My househ heart-so incorplated,
Or modest lovers' penuly vow'd up:
That reason thus shall stand at Pomfret. Lords, well
The valiant and opposeth tears:
This dead and honesty to answer them,
And from the neck of them shall pay for the least.

CLARENCE:

Ghost of, Nayler, my lords, kill me?
I have heard so much aloud ou

temp 1.5

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_15.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
He hath deserved clothes Plaimns with mine.'

AUFIDIUS:
God keep yourselves as speact sound
And feech the lip of such valour, let us murderous.

TRANIO:
A vengeance merry; let them respect.

TYBALT:
Falled them!

LADY GREY:
Why, then all served in any house;
Besides, and thenkering him and show'd;
an it hath twenty chaffing vow Juliet's death; no babled sons,
Being tenderly Heam Towards cale:
If this be nelectly.

DUKE VINCENTIO:
List. Go, cost I have had ere thee talk'd withal!

CATESBY:
Ay, my good lord; there is an air of mide
Than Edward freely. Go, safe; tempt bityers to secure and well,
Let me set not four difficer Kate to you
to put breath their guiltly newses, coming won;
The bast oranges right acted: alth, that
Apollo plucked that up will fought;
Awakes, nay, nay, my doom and London,
To see her bewartire heads but wave thy way;
Yick-brid came then my deserves at spates for Rome;
The banner so through at all to late,
To bear a park appray, and thereby the eastern

- temp 3

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['First Citizen:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_30.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

First Citizen:
He call'd me-loy,' slaughter'd resign,
Wroth, my sona, Wilker Earlt the powe, call-Piked.
So, underingrancely numbers were,
I would starely in the best from slue inJEnque for vein,
Constant thea!

Gaoutor:
Obey,
and melrown, this it his, Pemery and Lord!
Heveok-wife, Hencoin sta't; I knywife.
Now, obisably.

VolsaX
ADO:
Howal; t! andZGurst i' the Couctim-'s nobor?' daig, wied's paper?
Ah, Rarely's king!' Wef-dain!
Cede? hope, limis,ed Balk against their Gues:
ehe I that jest drybesh frammingbyiefds; igh the warcupal Wreth!
Ah, Wyry,t by urthwazen through us, bown, deserves,
Shalt Laybas TaPqoivis cups vigoward up; unchapquity,
Amb Puby's wit; my po! inhesiage and honority.
It is's, Poliant's awnqu: if CprostERBus!
Bring for, thought! and, FrameL, deadly Kate!
Did loudh'd yet again; a you!
Brag means
So, vio! Nay, besurry, to-morrow makest!
Give sort gots what I rove, hut, Boausen king!
Keep, wholesome-hearted friend no limit?
Wa wigodorts, bowsil'st you, Lucio!
Leaven lu

Guardo los pesos del entrenamiento

In [ ]:

folder_name = "/content/training_checkpoints/ckpt_54"
zip_name = "training_checkpoin.zip"

# Comprime la carpeta
!zip -r {zip_name} {folder_name}

  adding: content/training_checkpoints/ckpt_54/ (stored 0%)
  adding: content/training_checkpoints/ckpt_54/saved_model.pb (deflated 89%)
  adding: content/training_checkpoints/ckpt_54/keras_metadata.pb (deflated 78%)
  adding: content/training_checkpoints/ckpt_54/variables/ (stored 0%)
  adding: content/training_checkpoints/ckpt_54/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/training_checkpoints/ckpt_54/variables/variables.index (deflated 59%)
  adding: content/training_checkpoints/ckpt_54/assets/ (stored 0%)
  adding: content/training_checkpoints/ckpt_54/fingerprint.pb (stored 0%)


https://colab.research.google.com/github/FCEIA-AAII/lab10/blob/master/lab10-a.ipynb#scrollTo=ST7PSyk9t1mT

A pesar de tener un tiempo de ejecución bastante consistente al generar 1000 carácteres. Podemos observar que a menor temperatura la generación es más coherente y con poca creatividad. Ocurriendo lo contrario a medida que se aumenta la temperatura.

# Palabra

## Tokenización basada en palabras

Divide el texto en palabras únicas y crea los mapeos entre palabras e IDs.



In [ ]:
# Dividir el texto en palabras
words = tf.strings.split(text)
vocab = sorted(set(words.numpy()))  # Vocabulario único
print(f'{len(vocab)} unique words')

# Mapear palabras a índices y viceversa
words_from_ids = tf.keras.layers.StringLookup(vocabulary=vocab, invert=True, mask_token=None)
ids_from_words = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)

# Convertir el texto a IDs
all_ids = ids_from_words(words)


25670 unique words


## Preparación de datos para entrenamiento

Crea secuencias de palabras para entrenar el modelo y divide cada secuencia en entradas y objetivos.

In [ ]:
# Crear secuencias de palabras
seq_length = 50  # Longitud de las secuencias
sequences = tf.data.Dataset.from_tensor_slices(all_ids).batch(seq_length + 1, drop_remainder=True)

# Dividir secuencias en entrada y objetivo
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Configurar dataset para entrenamiento
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


## Definición del modelo basado en palabras

Crea un modelo con embeddings, una capa GRU y una capa densa para predecir palabras.

In [ ]:
# Configuración del modelo
vocab_size = len(vocab) + 1  # +1 para el índice 0 reservado
embedding_dim = 256
rnn_units = 1024

class WordModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        return (x, states) if return_state else x

# Crear instancia del modelo
word_based_model = WordModel(vocab_size, embedding_dim, rnn_units)


## Compilación y entrenamiento del modelo

Define la función de pérdida y entrena el modelo.

In [ ]:
# Directorio para guardar checkpoints
checkpoint_dir = './word_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
os.makedirs(checkpoint_dir, exist_ok=True)  # Crear el directorio si no existe

# Callback para guardar checkpoints
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=False,  # Solo guarda los pesos
    save_best_only=True,     # Guarda solo si es el mejor modelo hasta el momento
    monitor='loss',          # Monitorea la pérdida
    verbose=1
)

# Callback para detener el entrenamiento temprano
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',          # Monitorea la pérdida
    patience=5,              # Número de épocas sin mejora antes de detener
    restore_best_weights=True  # Restaura los pesos del mejor modelo
)


In [ ]:
# Función de pérdida
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compilar el modelo
word_based_model.compile(optimizer='adam', loss=loss)

# Entrenar el modelo
EPOCHS = 100


In [ ]:
# Entrenamiento con callbacks
history = word_based_model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

Epoch 1/100
62/62 [==============================] - ETA: 0s - loss: 6.7383
Epoch 1: loss improved from inf to 6.73830, saving model to ./word_training_checkpoints/ckpt_1
62/62 [==============================] - 20s 259ms/step - loss: 6.7383
Epoch 2/100
62/62 [==============================] - ETA: 0s - loss: 6.3323
Epoch 2: loss improved from 6.73830 to 6.33229, saving model to ./word_training_checkpoints/ckpt_2
62/62 [==============================] - 16s 256ms/step - loss: 6.3323
Epoch 3/100
62/62 [==============================] - ETA: 0s - loss: 6.0929
Epoch 3: loss improved from 6.33229 to 6.09291, saving model to ./word_training_checkpoints/ckpt_3
62/62 [==============================] - 16s 246ms/step - loss: 6.0929
Epoch 4/100
62/62 [==============================] - ETA: 0s - loss: 5.8066
Epoch 4: loss improved from 6.09291 to 5.80655, saving model to ./word_training_checkpoints/ckpt_4
62/62 [==============================] - 16s 243ms/step - loss: 5.8066
Epoch 5/100
62/62 [=

## Generador de texto basado en palabras

Define una clase para generar texto palabra por palabra.

In [ ]:
class WordGenerator(tf.keras.Model):
    def __init__(self, model, words_from_ids, ids_from_words, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.words_from_ids = words_from_ids
        self.ids_from_words = ids_from_words

    def generate_words(self, seed_text, num_words=50, states=None):
      # Convertir las palabras de entrada en IDs
      input_ids = self.ids_from_words(tf.strings.split(seed_text))
      input_ids = tf.expand_dims(input_ids, axis=0)  # Agregar dimensión para el batch

      result = [seed_text]  # Lista de palabras generadas, comenzando con el texto inicial

      for _ in range(num_words):
          # Obtener las predicciones del modelo
          predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
          # Ajustar las probabilidades según la temperatura
          predicted_logits = predicted_logits[:, -1, :] / self.temperature
          # Generar una palabra basada en las probabilidades ajustadas
          predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
          predicted_ids = tf.squeeze(predicted_ids, axis=-1)

          # Convertir el ID predicho en la palabra correspondiente
          predicted_word = self.words_from_ids(predicted_ids).numpy()

          # Si `predicted_word` es un arreglo, convertir cada elemento a str
          if isinstance(predicted_word, np.ndarray):
              predicted_word = predicted_word[0].decode('utf-8')  # Decodificar el primer elemento

          result.append(predicted_word)

          # Actualizar el input para la próxima iteración
          input_ids = tf.expand_dims(predicted_ids, axis=0)

      # Unir las palabras generadas en un texto
      return ' '.join(result)

## Generación de texto

Usa el generador de texto para crear nuevas secuencias basadas en palabras.

In [ ]:
# Instanciar el generador de texto con el modelo entrenado
word_gen_model = WordGenerator(word_based_model, words_from_ids, ids_from_words, temperature=0.5)

# Generar texto
seed_text = "To be or not to be"
generated_text = word_gen_model.generate_words(seed_text=seed_text, num_words=50)
print(f"Texto generado con temperatura 0.5:\n{generated_text}")

# Experimentar con otra temperatura
word_gen_model_high_temp = WordGenerator(word_based_model, words_from_ids, ids_from_words, temperature=1.5)
generated_text_high_temp = word_gen_model_high_temp.generate_words(seed_text=seed_text, num_words=50)
print(f"Texto generado con temperatura 1.5:\n{generated_text_high_temp}")


Texto generado con temperatura 0.5:
To be or not to be a king, As poisonous than a poor traitor to instruct your sword, Which came becomes your sword, and gentle wind May instruct your shame, But mercy to be your day we do hold your daughter and your purpose to your shame, And then, and reconcile us your fortune side and
Texto generado con temperatura 1.5:
To be or not to be a king, That did upon me: if his words might fall you that my hand. Shepherd: Never? and wrathful no, you must not speak with us? I am your queen, with Licio. LUCIO: When you have been Me for, another's Above the best of war, ourselves to his purpose and


Como ya habíamos concluido, es notable que la temperatura hace una modificación en la creatividad y coherencia en la generación de texto.
Siendo que al tener temperaturas bajas, obtenemos resultados más seguros, coherentes, estructuras que se va perdiendo al ir aumentando la temperatura.